# Component contribution analysis

From the hyperparameter tuning step, we have selected a set of parameters ($\lambda$,  $ \vartheta$, k) with largest **LLPD$_o$**.  

Here we aim to evaluate the contribution of each of the components in the generative model (MEM). The generative model for the ocean microbiome data for the microbial abundance data consists of **geochemical component**, **spatio-temporal components indicating geographical location, ocean depth and time** and **species-species interaction component**. To understand their  contribution in the MEM, we drop each of the specific component in the generative model, estimate the variational posterior and then evaluate the model performance in terms of the **out of sample log-likelihood predictive density (LLPD$_o$)**.


Steps of the analysis
 + Define component excluded generative model
 + Python script for the variational posterior estimation: **component_contribution_fit.py**
 + Script to call component excluded generative model  
 + Evaluate the model in terms of the **LLPD$_o$**

 
#### Component excluded generative model
We have defined the component excluded stan model (see stan_model folder) in the following files:
 + **NB_microbe_ppc.stan** : Full model [Model = 0]
 + **NB_microbe_ppc-1.stan** : Province component dropped  [Model = 1]
 + **NB_microbe_ppc-2.stan** : Biome component dropped  [Model = 2]
 + **NB_microbe_ppc-3.stan** : Quarter/Time component dropped  [Model = 3]
 + **NB_microbe_ppc-G.stan** : Geochemical component dropped  [Model = 4]
 + **NB_microbe_ppc_nointer.stan** : Species-species interaction component dropped  [Model = 5]

#### Script to evaluate the model
For the chosen set of hyperparameters, we compute the variational posterior of each the **component excluded generative model** for **twenty** different initialization. We have saved the command calling **component_contribution_fit.py** for each of the  **component excluded generative model** in the file **mem_component_contribution** (see the python cript below to generate the file).

A line in the file **mem_component_contribution** calls the python script **component_contribution_fit.py** for a given choice of the parameters. 

*module purge ; module load slurm gcc python3 ; OMP_NUM_THREADS=1 python3 component_contribution_fit.py 100.0 0.219 0.06503 0.1 200 1 0 1.0 > logfile/1.log 2>&1*

Input parameters for a setting includes **latent rank (k),  $\lambda$,  $\vartheta$, test sample proportion, variational posterior sample size,  sub-settings unique ID (1-20), model type  and setting seed**. 


#### Parameter estimation 
We run the script on server using the command:
*sbatch -N [#node] -p [#partition] disBatch.py -t [#task on each node] [script_file]*
Example: *sbatch -N 10 -p ccm disBatch.py -t 25 mem_component_contribution*




#### Model output analysis
Evaluate each of the **component excluded generative model** based on the $LLPD_o$. Let us consider our model output is saved in the folder **CContribution**. To compare the model, we load the output file and compute the $LLPD_o$ for each model. 

In [1]:
# load python module 
import os
import numpy as np
import random
import pickle as pkl
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# output file name 
# We have save the output in [CContribution] folder
fname_o = glob.glob('../results/component/models/*model_nb_cvtest.pkl') 
out = np.empty((len(fname_o),7))
mtype_file = np.argsort([int(x.split('/')[1].split('_')[1].split('.')[0]) for x in fname_o])
fname_o = [fname_o[i] for i in mtype_file]
#fname_o[2]

In [ ]:
# Extract model output and compute LLPD
for i in range(len(fname_o)):
    if ((i%20) == 0):
        print(i)
    [holdout_mask, Yte_sample, llpd, n_test, l,m_seed,sp_mean,\
                 sp_var, h_prop, uid, nsample_o, Yte_fit,\
                 cv_test,Y, muest, Yte_cv, Yte_lpmf, kl_comp] = pkl.load(open(fname_o[i], "rb"))
    se_index  = holdout_mask == 1. 

    # LLPD compute 
    temp_ll = cv_test[se_index]
    temp_ll = np.mean(temp_ll)  
    mtype = fname_o[i].split('/')[1].split('_')[1]
    mtype = int(mtype.split('.')[0])
    error = np.mean(np.power(Y - muest,2)[se_index])
    out[i] = [i, l, sp_mean,sp_var,  temp_ll,uid, mtype]


In [4]:
pkl.dump(out, open('comparison_model', "wb"))  # save output 
out = pkl.load(open('comparison_model', "rb"))
out = pd.DataFrame(data=out)
out.columns = ['Index','rank','lambda','upsilon','llpd','uid','Model']
out.head(20)

,Index,rank,lambda,upsilon,llpd,uid,Model


<font color=blue>**Our analysis suggest that the model 0, i.e., MEM is best with highest out of sample LLPD$_o$.** </font>

In [ ]:
out.groupby(['Model'], as_index=False).agg({'llpd':['mean','std']})

Empty DataFrame
Columns: [(Model, ), (llpd, mean), (llpd, std)]
Index: []

In [6]:
out_sub = out.groupby(['Model'], as_index=False).mean()[['Model','llpd']].transpose()
print(out_sub.to_latex(float_format="%.4f", bold_rows = True))

\begin{tabular}{l}
\toprule
\textbf{Empty DataFrame
Columns: Int64Index([], dtype='int64')
Index: Index(['Model', 'llpd'], dtype='object')} \\
\bottomrule
\end{tabular}



In [7]:
l,m_seed,sp_mean, sp_var

NameError: name 'l' is not defined